In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
train_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv',nrows = 1000000)
test_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')


In [ ]:
train_df


In [ ]:
test_df


In [ ]:
train_df.columns


In [ ]:
test_df.columns


In [ ]:
train_df.isnull().sum().sort_values(ascending=False)


In [ ]:
test_df.isnull().sum().sort_values(ascending=False)


In [ ]:
a = train_df[train_df["dropoff_longitude"].isnull()|train_df["dropoff_latitude"].isnull()]
print(a)


In [ ]:
train_df.drop(a.index,axis=0, inplace=True)


In [ ]:
train_df.isnull().sum().sort_values(ascending=False)


In [ ]:
 train_df.describe()


In [ ]:
test_df.describe()


In [ ]:
sns.histplot(
data = train_df, x = "fare_amount", kde = True, bins = 100)


In [ ]:
sns.boxplot(
data = train_df, y = "fare_amount")


In [ ]:
train_df['passenger_count'].describe()


In [ ]:
sns.histplot(
data = train_df, x = "passenger_count")
plt.ylim(0,1000000)


In [ ]:
train_df = train_df[(train_df["fare_amount"] >= 2.5) & (train_df["fare_amount"] < 100)]


In [ ]:
train_df["fare_amount"].describe()


In [ ]:
sns.histplot(
data = train_df, x = "fare_amount", kde = True, bins = 100 )
plt.xlim(0,80)
plt.ylim(0,150000)


In [ ]:
sns.boxplot(
data = train_df, y = "fare_amount")


In [ ]:
train_df = train_df[(train_df["passenger_count"] <= 6) & (train_df["passenger_count"] >= 1)]
sns.histplot(
data = train_df, x = "passenger_count")
plt.ylim(0,1000000)


In [ ]:
train_df = train_df[(train_df["pickup_longitude"] <= -73.0) & (train_df["pickup_longitude"] >= -74.5)]
train_df = train_df[(train_df["pickup_latitude"] >= 40.5) & (train_df["pickup_latitude"] <= 42)]
train_df = train_df[(train_df["dropoff_longitude"] <= -73.0) & (train_df["dropoff_longitude"] >= -74.5)]
train_df = train_df[(train_df["dropoff_latitude"] >= 40.5) & (train_df["dropoff_latitude"] <= 42)]
## 42~40.5,-74.5~-73


In [ ]:
train_df.describe()


In [ ]:
train_df


In [ ]:
train_df.describe()


In [ ]:
train_df.drop(['key','pickup_datetime'], axis = 1, inplace=True)
test_df.drop(['key','pickup_datetime'], axis = 1, inplace=True)


In [ ]:
train_df.columns


In [ ]:
test_df.columns


In [ ]:
from geopy.distance import geodesic
def get_distance_km(lat1, lon1, lat2, lon2):
    p_point = (lat1, lon1)
    d_point = (lat2, lon2)
    distance_km = geodesic(p_point, d_point).km
    return distance_km


In [ ]:
train_len = len(train_df)
print(train_len)


In [ ]:
df = pd.concat([train_df,test_df], axis=0)


In [ ]:
print(len(df))


In [ ]:
distances_km = []
for i in range(len(df)):
    distance = get_distance_km(df.iloc[i,df.columns.get_loc("pickup_latitude")],df.iloc[i,df.columns.get_loc("pickup_longitude")],df.iloc[i,df.columns.get_loc("dropoff_latitude")],df.iloc[i,df.columns.get_loc("dropoff_longitude")])
    distances_km.append(distance)


In [ ]:
print(distances_km[:20])


In [ ]:
import statistics
mean = statistics.mean(distances_km)
print(mean)


In [ ]:
df["distance"] = distances_km


In [ ]:
df


In [ ]:
train = df[0:train_len]
test = df[train_len:]


In [ ]:
train.isnull().sum().sort_values(ascending=False)


In [ ]:
test.isnull().sum().sort_values(ascending=False)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [ ]:
train_X = train.drop("fare_amount",axis=1).values
train_y = train["fare_amount"].values
test = test.drop("fare_amount",axis=1).values
train_x, test_x, train_y, test_y = train_test_split(train_X,train_y,test_size=0.3,random_state=0)


In [ ]:
model = LinearRegression()
model.fit(train_x,train_y)
test_x_predict = model.predict(test_x)


In [ ]:
print("score:"+str(np.sqrt(mean_squared_error(test_y,test_x_predict))))


In [ ]:
sub = pd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv")
sub["fare_amount"] = model.predict(test)


In [ ]:
sub.to_csv("submission.csv",index=False)


In [ ]:
sub
